In [1]:
import sys
import numpy as np
sys.path.append('../')
from time import sleep
from quantum_compiler import shape_functionV4 as sf
from quantum_compiler.WavetoolsV6 import Wave
from instruments.core.jsonIO import jsonFormat
from instruments.drivers.Tektronix.Tektronix_AWG import AWG5208

In [2]:
# connect to AWG
awg = AWG5208(
    inst_name='AWG5208',
    inst_address='TCPIP0::192.168.10.42::inst0::INSTR')

# help(awg)

# set sampling rate and reference clock
awg.set_sample_rate(sample_rate=1.0E9)
awg.set_extref_source(ref_freq=10E6)

Connect Instrument:
instrument: AWG5208
statement: TEKTRONIX,AWG5208,B030597,FV:6.6.0131.0


In [3]:
# awg.save_inst_cfg()

In [4]:
sample_rate=1.0E9
x_span=5e-6
start=1e-6
flat=0.4e-6
time_range = np.linspace(0, 500e-9, 51)
IQList = ['ch1', 'ch2']


marker = ~Wave(sf.square, [sf.get_x(x_span, sample_rate), 0.1e-6, 0.1e-6])
square2 = ~Wave(sf.gaussian_square, [sf.get_x(x_span, sample_rate), start, flat, 10e-9])

# clear, set, and upload
awg.clr_wfm()
for time in time_range:
    square = ~Wave(sf.gaussian_square, [sf.get_x(x_span, sample_rate), start-time, time, 10e-9])

    awg.set_wfm(wfm_name=f'{IQList[0]}_{time}',
            wfm=(square).y,
            mkr1=(marker).y,
            mkr2=(marker).y
           )
    awg.set_wfm(wfm_name=f'{IQList[1]}_{time}',
        wfm=(square2).y
       )
awg.upload_wfm()

In [5]:
# assign waveform to corresponding sequence position one at a time
seq_name='freq_mod_iq'
trackList = IQList

# clear, set, assign and upload
awg.clr_seq()
awg.set_seq(seq_name, num_track=len(trackList), num_step=len(time_range))
for track_idx, track in enumerate(trackList):
    for step_idx, time in enumerate(time_range):
        awg.assign_seq(f'{track}_{time}', seq_name, track_idx+1, step_idx+1)
awg.upload_seq(seq_name)

In [6]:
# assign waveforms/sequences to channels
awg.assign_ch(1, 'freq_mod_iq', track_index=1)
awg.assign_ch(2, 'freq_mod_iq', track_index=2)
awg.assign_ch(3, 'ch2_0.0', auto_output=False, auto_mkr=False)